# Homework 1 Ahmet Akman
This notebook will consist the efforts led by homework 1 and done by Ahmet Akman in the scope of EE449 Course

## Imports

In [ ]:
import numpy as np
import sklearn
import torch
import matplotlib.pyplot as plt

## Basic Neural Network Construction and Training


### 1.1




In [ ]:
x = np.arange(-2, 2, 0.01, dtype = float)
tanh_result = (np.exp(2*x)-1) / (np.exp(2*x)+1)
sigmoid_result =  1 / (np.exp(-x)+1)
ReLU_result = np.maximum(0,x)


fig, ax = plt.subplots(3)
ax[0].plot(x,tanh_result)
ax[0].set_title("Tanh function")
ax[0].set_xlabel("x")
ax[0].set_ylabel("tanh(x)")
ax[0].grid()

ax[1].plot(x,sigmoid_result)
ax[1].set_title("Sigmoid function")
ax[1].set_xlabel("x")
ax[1].set_ylabel("sigma(x)")
ax[1].grid()

ax[2].plot(x,ReLU_result)
ax[2].set_title("ReLU function")
ax[2].set_xlabel("x")
ax[2].set_ylabel("ReLU(x)")
ax[2].grid()
fig.tight_layout()


In [ ]:
p_tanh_result = 1- (np.exp(x)-np.exp(-x))**2 / (np.exp(x) + np.exp(-x))**2
p_sigmoid_result =  np.exp(-x) / (np.exp(-x)+1)**2
p_ReLU_result = x >= 0 


fig, ax = plt.subplots(3)
ax[0].plot(x, p_tanh_result)
ax[0].set_title("Partial derivative of Tanh function")
ax[0].set_xlabel("x")
ax[0].set_ylabel("tanh(x)")
ax[0].grid()

ax[1].plot(x, p_sigmoid_result)
ax[1].set_title("Partial derivative of Sigmoid function")
ax[1].set_xlabel("x")
ax[1].set_ylabel("sigma(x)")
ax[1].grid()

ax[2].plot(x, p_ReLU_result)
ax[2].set_title("Partial derivative of ReLU function")
ax[2].set_xlabel("x")
ax[2].set_ylabel("ReLU(x)")
ax[2].grid()
fig.tight_layout()

### 1.2
- Sigmoid activated MLP.


In [ ]:
class MLP_sigmoid:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        self.bias_output = np.zeros((1, self.output_size))

    def sigmoid(self, x):
        return np.exp(x) / (np.exp(x)+1)
    def sigmoid_derivative(self, x):
        return np.exp(-x) / (np.exp(-x)+1)**2

    def forward(self, inputs):
        # Forward pass through the network
        self.hidden_output =  self.sigmoid(np.dot(inputs, self.weights_input_hidden) + self.bias_hidden)
        self.output =  np.round(np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output)
        return self.output
    
    def backward(self, inputs, targets, learning_rate):
        # Backward pass through the network
        # Compute error
        output_error = targets - self.output
        hidden_error = output_error * self.sigmoid_derivative(self.hidden_output)
        # Compute gradients
        output_delta = output_error * self.sigmoid_derivative(self.output)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden_output)
        # Update weights and biases
        self.weights_hidden_output = self.weights_hidden_output + learning_rate * np.dot(self.hidden_output.T, output_delta)
        self.bias_output = self.bias_output + learning_rate * np.sum(output_delta, axis=0, keepdims=True)
        self.weights_input_hidden = self.weights_input_hidden + learning_rate * np.dot(inputs.T, hidden_delta)
        self.bias_hidden = self.bias_hidden + learning_rate * np.sum(hidden_delta, axis=0, keepdims=True)

from utils import part1CreateDataset, part1PlotBoundary
x_train, y_train, x_val, y_val = part1CreateDataset(train_samples=1000, val_samples=100, std=0.4)

# Define neural network parameters
input_size = 2
hidden_size = 4
output_size = 1
learning_rate = 0.0001
# Create neural network
nn = MLP_sigmoid(input_size, hidden_size, output_size)
# Train the neural network
for epoch in range(10000):
    # Forward propagation
    output = nn.forward(x_train)
    # Backpropagation
    nn.backward(x_train, y_train, learning_rate)

    # Print the loss (MSE) every 1000 epochs
    if epoch % 1000 == 0:
        loss = np.mean((y_train - output)**2)
        print(f'Epoch {epoch}: Loss = {loss}')

        
# Test the trained neural network
y_predict = nn.forward(x_val)
print(f'{np.mean(y_predict==y_val)*100} % of test examples classified correctly.')


part1PlotBoundary(x_val, y_val, nn)

- Tanh activated MLP. # TODO

In [ ]:
class MLP_tanh:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        self.bias_output = np.zeros((1, self.output_size))

    def tanh(self, x):
        return (np.exp(2*x)-1) / (np.exp(2*x)+1)
    def tanh_derivative(self, x):
        return 1- (np.exp(x)-np.exp(-x))**2 / (np.exp(x) + np.exp(-x))**2

    def forward(self, inputs):
        # Forward pass through the network
        self.hidden_output =  self.tanh(np.dot(inputs, self.weights_input_hidden) + self.bias_hidden)
        self.output =  np.round(np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output)
        return self.output
    
    def backward(self, inputs, targets, learning_rate):
        # Backward pass through the network
        # Compute error
        output_error = targets - self.output
        hidden_error = output_error * self.tanh_derivative(self.hidden_output)
        # Compute gradients
        output_delta = output_error * self.tanh_derivative(self.output)
        hidden_delta = hidden_error * self.tanh_derivative(self.hidden_output)
        # Update weights and biases
        self.weights_hidden_output = self.weights_hidden_output + learning_rate * np.dot(self.hidden_output.T, output_delta)
        self.bias_output = self.bias_output + learning_rate * np.sum(output_delta, axis=0, keepdims=True)
        self.weights_input_hidden = self.weights_input_hidden + learning_rate * np.dot(inputs.T, hidden_delta)
        self.bias_hidden = self.bias_hidden + learning_rate * np.sum(hidden_delta, axis=0, keepdims=True)

from utils import part1CreateDataset, part1PlotBoundary
x_train, y_train, x_val, y_val = part1CreateDataset(train_samples=1000, val_samples=100, std=0.4)

# Define neural network parameters
input_size = 2
hidden_size = 4
output_size = 1
learning_rate = 0.00001
# Create neural network
nn = MLP_tanh(input_size, hidden_size, output_size)
# Train the neural network
for epoch in range(10000):
    # Forward propagation
    output = nn.forward(x_train)
    # Backpropagation
    nn.backward(x_train, y_train, learning_rate)

    # Print the loss (MSE) every 1000 epochs
    if epoch % 1000 == 0:
        loss = np.mean((y_train - output)**2)
        print(f'Epoch {epoch}: Loss = {loss}')

        
# Test the trained neural network
y_predict = nn.forward(x_val)
print(f'{np.mean(y_predict==y_val)*100} % of test examples classified correctly.')


part1PlotBoundary(x_val, y_val, nn)

- ReLU Activated MLP


In [ ]:
class MLP_ReLU:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights and biases
        self.weights_input_hidden = np.random.randn(self.input_size, self.hidden_size)
        self.bias_hidden = np.zeros((1, self.hidden_size))
        self.weights_hidden_output = np.random.randn(self.hidden_size, self.output_size)
        self.bias_output = np.zeros((1, self.output_size))

    def ReLU(self, x):
        return np.maximum(0,x)
    def ReLU_derivative(self, x):
        return x >= 0

    def forward(self, inputs):
        # Forward pass through the network
        self.hidden_output =  self.ReLU(np.dot(inputs, self.weights_input_hidden) + self.bias_hidden)
        self.output =  np.round(np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output)
        return self.output
    
    def backward(self, inputs, targets, learning_rate):
        # Backward pass through the network
        # Compute error
        output_error = targets - self.output
        hidden_error = output_error * self.ReLU_derivative(self.hidden_output)
        # Compute gradients
        output_delta = output_error * self.ReLU_derivative(self.output)
        hidden_delta = hidden_error * self.ReLU_derivative(self.hidden_output)
        # Update weights and biases
        self.weights_hidden_output = self.weights_hidden_output + learning_rate * np.dot(self.hidden_output.T, output_delta)
        self.bias_output = self.bias_output + learning_rate * np.sum(output_delta, axis=0, keepdims=True)
        self.weights_input_hidden = self.weights_input_hidden + learning_rate * np.dot(inputs.T, hidden_delta)
        self.bias_hidden = self.bias_hidden + learning_rate * np.sum(hidden_delta, axis=0, keepdims=True)

from utils import part1CreateDataset, part1PlotBoundary
x_train, y_train, x_val, y_val = part1CreateDataset(train_samples=1000, val_samples=100, std=0.4)

# Define neural network parameters
input_size = 2
hidden_size = 4
output_size = 1
learning_rate = 0.00001
# Create neural network
nn = MLP_ReLU(input_size, hidden_size, output_size)
# Train the neural network
for epoch in range(10000):
    # Forward propagation
    output = nn.forward(x_train)
    # Backpropagation
    nn.backward(x_train, y_train, learning_rate)

    # Print the loss (MSE) every 1000 epochs
    if epoch % 1000 == 0:
        loss = np.mean((y_train - output)**2)
        print(f'Epoch {epoch}: Loss = {loss}')

        
# Test the trained neural network
y_predict = nn.forward(x_val)
print(f'{np.mean(y_predict==y_val)*100} % of test examples classified correctly.')


part1PlotBoundary(x_val, y_val, nn)

### 1.3


## Implementing a Convolutional Layer with NumPy

In [ ]:


# implement a 2D convolutional layer using numpy
def my_conv2d(input, kernel):
    # input shape: [batch size, input_channels, input_height, input_width]
    # kernel shape: [output_channels, input_channels, filter_height, filter width]
    batch_size, input_channels, input_height, input_width = input.shape
    output_channels, input_channels, filter_height, filter_width = kernel.shape
    # output shape: [batch size, output_channels, output_height, output_width]
    output_height = input_height - filter_height + 1
    output_width = input_width - filter_width + 1
    output = np.zeros((batch_size, output_channels, output_height, output_width))
    for b in range(batch_size):
        for oc in range(output_channels):
            for ic in range(input_channels):
                for i in range(output_height):
                    for j in range(output_width):
                        output[b, oc, i, j] = np.sum(input[b, ic, i:i+filter_height, j:j+filter_width] * kernel[oc, ic])
    return output

# input shape: [batch size, input_channels, input_height, input_width]
kernel=np.load('data/kernel.npy')

for i in range(10):
    input=np.load('data/samples_{}.npy'.format(i))
    # input shape: [output_channels, input_channels, filter_height, filter width]
    out = my_conv2d(input, kernel)
    out_check = torch.conv2d(torch.tensor(input).float(), torch.tensor(kernel).float())
    np.save('outputs/out_{}.npy'.format(i), out)

    from utils import part2Plots
    part2Plots(out = out, save_dir='outputs', filename='out_{}'.format(i) )
    part2Plots(out = out_check, save_dir='outputs', filename='out_{}_check'.format(i) )

## 2.2


In [ ]:
%reset -f

## Experimenting ANN Architectures

In [ ]:
# Load fashion MNIST dataset
import torchvision

#training set
train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False, transform= torchvision.transforms.ToTensor())

#test set
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False, transform= torchvision.transforms.ToTensor())


In [ ]:
import torch
# divide training data into training and validation sets of 0.8 and 0.2 respectively

train_data, val_data = torch.utils.data.random_split(train_data, [0.8, 0.2])

train_generator = torch.utils.data.DataLoader(train_data, batch_size=96, shuffle=True)
val_generator = torch.utils.data.DataLoader(val_data, batch_size=96, shuffle=False)
test_generator = torch.utils.data.DataLoader(test_data, batch_size=96, shuffle=False)


### mlp_1 case
mlp_1 corresponds to FC-32, ReLU + FC10

In [ ]:


# example mlp classifier
class mlp_1(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(mlp_1, self).__init__()
        self.input_size = input_size
        self.FC = torch.nn.Linear(input_size, hidden_size)
        self.prediction_layer = torch.nn.Linear(hidden_size, num_classes)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(-1, self.input_size)
        hidden = self.FC(x)
        relu = self.relu(hidden)
        output = self.prediction_layer(relu)
        return output

# initialize your model
model_mlp_1 = mlp_1(784,32,10)
if torch.cuda.is_available():
    model_mlp_1 = model_mlp_1.cuda()

# create loss: use cross entropy loss
loss = torch.nn.CrossEntropyLoss()
# create optimizer
optimizer = torch.optim.Adam(model_mlp_1.parameters(), lr=0.001)
# transfer your model to train mode
model_mlp_1.train()


mlp_1_dict = {"name":"mlp_1", "loss_curve": [],  "train_acc_curve": [], "val_acc_curve": [], "test_acc": 0, 'weights': []}



# train the model and save the training loss and validation loss for every 10 batches using model eval mode.s
for epoch in range(15):
    for batch, (x, y) in enumerate(train_generator):
        # forward pass
        output = model_mlp_1(x)
        # compute loss
        loss_val = loss(output, y)
        # zero gradients
        optimizer.zero_grad()
        # backward pass
        loss_val.backward()
        # optimize
        optimizer.step()
        # print loss
        if batch % 10 == 0:
            model_mlp_1.eval()
            #print('Epoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, loss_val.item()))
            mlp_1_dict['loss_curve'].append(loss_val.item())
            training_accuracy = torch.mean((torch.argmax(output, dim=1) == y).float())
            mlp_1_dict['train_acc_curve'].append(training_accuracy.item())

            # validation loss   
            validation_accuracy_per_batch = torch.tensor([])
            with torch.no_grad():
                for val_x, val_y in val_generator:
                    val_output = model_mlp_1(val_x)
                    #val_loss = loss(val_output, val_y)
                    val_accuracy = torch.mean((torch.argmax(val_output, dim=1) == val_y).float())
                    try:
                        validation_accuracy_per_batch = torch.cat((validation_accuracy_per_batch ,val_accuracy))
                    except:
                        validation_accuracy_per_batch = val_accuracy
            #print('Validation Accuracy: {}'.format(validation_accuracy_per_batch.mean()))
            mlp_1_dict['val_acc_curve'].append(validation_accuracy_per_batch.mean().item())
            model_mlp_1.train()



# test the model
correct = 0
total = 0
for x, y in test_generator:
    output = model_mlp_1(x)
    _, predicted = torch.max(output, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print('Test Accuracy: {} %'.format(100 * correct / total))
mlp_1_dict['test_acc'] = 100 * correct / total
# save the model as a pty file
torch.save(model_mlp_1.state_dict(), 'q3_models/model_mlp_1.pty')
print("model saved as 'q3_models/model_mlp_1.pty'")
# get the parameters 784x32 layer as numpy array
weights_first_layer = model_mlp_1.FC.weight.data.numpy()
mlp_1_dict['weights'] = weights_first_layer


In [ ]:
# save mlp_1_dict as a pickle file
import pickle
with open('q3_models/mlp_1_dict.pkl', 'wb') as f:
    pickle.dump(mlp_1_dict, f)


### mlp_2 case


In [ ]:
%reset -f

# Load fashion MNIST dataset
import torchvision

from tqdm import tqdm

#training set
train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False, transform= torchvision.transforms.ToTensor())

#test set
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False, transform= torchvision.transforms.ToTensor())


import torch
# divide training data into training and validation sets of 0.8 and 0.2 respectively

train_data, val_data = torch.utils.data.random_split(train_data, [0.8, 0.2])

train_generator = torch.utils.data.DataLoader(train_data, batch_size=96, shuffle=True)
val_generator = torch.utils.data.DataLoader(val_data, batch_size=96, shuffle=False)
test_generator = torch.utils.data.DataLoader(test_data, batch_size=96, shuffle=False)





# example mlp classifier
class mlp_2(torch.nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, num_classes):
        super(mlp_2, self).__init__()
        self.input_size = input_size
        self.FC1 = torch.nn.Linear(input_size, hidden_size_1)
        self.FC2 = torch.nn.Linear(hidden_size_1, hidden_size_2, bias=False)
        self.prediction_layer = torch.nn.Linear(hidden_size_2, num_classes)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(-1, self.input_size)
        hidden1 = self.FC1(x)
        relu = self.relu(hidden1)
        hidden2 = self.FC2(relu)
        output = self.prediction_layer(hidden2)
        return output

# initialize your model
model_mlp_2 = mlp_2(784 ,32 ,64 ,10)
if torch.cuda.is_available():
    model_mlp_2 = model_mlp_2.cuda()


# create loss: use cross entropy loss
loss = torch.nn.CrossEntropyLoss()
# create optimizer
optimizer = torch.optim.Adam(model_mlp_2.parameters(), lr=0.001)
# transfer your model to train mode
model_mlp_2.train()


mlp_2_dict = {"name":"mlp_2", "loss_curve": [],  "train_acc_curve": [], "val_acc_curve": [], "test_acc": 0, 'weights': []}



# train the model and save the training loss and validation loss for every 10 batches using model eval mode.s
for epoch in tqdm(range(15)):
    for batch, (x, y) in enumerate(train_generator):
        # forward pass
        output = model_mlp_2(x)
        # compute loss
        loss_val = loss(output, y)
        # zero gradients
        optimizer.zero_grad()
        # backward pass
        loss_val.backward()
        # optimize
        optimizer.step()
        # print loss
        if batch % 10 == 0:
            model_mlp_2.eval()
            #print('Epoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, loss_val.item()))
            mlp_2_dict['loss_curve'].append(loss_val.item())
            training_accuracy = torch.mean((torch.argmax(output, dim=1) == y).float())
            mlp_2_dict['train_acc_curve'].append(training_accuracy.item())

            # validation loss   
            validation_accuracy_per_batch = torch.tensor([])
            with torch.no_grad():
                for val_x, val_y in val_generator:
                    val_output = model_mlp_2(val_x)
                    #val_loss = loss(val_output, val_y)
                    val_accuracy = torch.mean((torch.argmax(val_output, dim=1) == val_y).float())
                    try:
                        validation_accuracy_per_batch = torch.cat((validation_accuracy_per_batch ,val_accuracy))
                    except:
                        validation_accuracy_per_batch = val_accuracy
            #print('Validation Accuracy: {}'.format(validation_accuracy_per_batch.mean()))
            mlp_2_dict['val_acc_curve'].append(validation_accuracy_per_batch.mean().item())
            model_mlp_2.train()



# test the model
correct = 0
total = 0
for x, y in test_generator:
    output = model_mlp_2(x)
    _, predicted = torch.max(output, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print('Test Accuracy: {} %'.format(100 * correct / total))
mlp_2_dict['test_acc'] = 100 * correct / total
# save the model as a pty file
torch.save(model_mlp_2.state_dict(), 'q3_models/model_mlp_2.pty')
print("model saved as 'q3_models/model_mlp_2.pty'")
# get the parameters 784x32 layer as numpy array
weights_first_layer = model_mlp_2.FC1.weight.data.numpy()
mlp_2_dict['weights'] = weights_first_layer


# save mlp_2_dict as a pickle file
import pickle
with open('q3_models/mlp_2_dict.pkl', 'wb') as f:
    pickle.dump(mlp_2_dict, f)

### CNN3
Where Conv - 3x3x16 Relu
Conv - 5x5x8 Relı, MaxPool 2x2
Conv - 7x7x16 MaxPool 2x2
Fully connected layer of 10.

In [ ]:
%reset -f

# Load fashion MNIST dataset
import torchvision

from tqdm import tqdm

#training set
train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False, transform= torchvision.transforms.ToTensor())

#test set
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False, transform= torchvision.transforms.ToTensor())


import torch
# divide training data into training and validation sets of 0.8 and 0.2 respectively

train_data, val_data = torch.utils.data.random_split(train_data, [0.8, 0.2])

train_generator = torch.utils.data.DataLoader(train_data, batch_size=96, shuffle=True)
val_generator = torch.utils.data.DataLoader(val_data, batch_size=96, shuffle=False)
test_generator = torch.utils.data.DataLoader(test_data, batch_size=96, shuffle=False)





# example cnn_3 classifier
class cnn_3(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(cnn_3, self).__init__()
        self.input_size = input_size
        self.Conv1 = torch.nn.Conv2d(1 ,16 ,3, stride=1, padding=1)
        self.MaxPool = torch.nn.MaxPool2d(2, stride=2)
        self.Conv2 = torch.nn.Conv2d(16, 8, 5,  stride=1, padding=1)
        self.Conv3 = torch.nn.Conv2d(8, 16, 7, stride=1, padding=1)
        self.prediction_layer = torch.nn.Linear(1296, num_classes)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        hidden1 = self.Conv1(x)
        relu1 = self.relu(hidden1)
        hidden2 = self.Conv2(relu1)
        relu2 = self.relu(hidden2)
        pool = self.MaxPool(relu2)
        hidden3 = self.Conv3(pool)
        flattened = hidden3.view(96, -1)
        output = self.prediction_layer(flattened)
        return output

# initialize your model
model_cnn_3 = cnn_3(784 ,10)
if torch.cuda.is_available():
    model_cnn_3 = model_cnn_3.cuda()


# create loss: use cross entropy loss
loss = torch.nn.CrossEntropyLoss()
# create optimizer
optimizer = torch.optim.Adam(model_cnn_3.parameters(), lr=0.001)
# transfer your model to train mode
model_cnn_3.train()


cnn_3_dict = {"name":"cnn_3", "loss_curve": [],  "train_acc_curve": [], "val_acc_curve": [], "test_acc": 0, 'weights': []}



# train the model and save the training loss and validation loss for every 10 batches using model eval mode.s
for epoch in tqdm(range(15)):
    for batch, (x, y) in enumerate(train_generator):
        # forward pass
        output = model_cnn_3(x)
        # compute loss
        loss_val = loss(output, y)
        # zero gradients
        optimizer.zero_grad()
        # backward pass
        loss_val.backward()
        # optimize
        optimizer.step()
        # print loss
        if batch % 10 == 0:
            model_cnn_3.eval()
            #print('Epoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, loss_val.item()))
            cnn_3_dict['loss_curve'].append(loss_val.item())
            training_accuracy = torch.mean((torch.argmax(output, dim=1) == y).float())
            cnn_3_dict['train_acc_curve'].append(training_accuracy.item())

            # validation loss   
            validation_accuracy_per_batch = torch.tensor([])
            with torch.no_grad():
                for val_x, val_y in val_generator:
                    val_output = model_cnn_3(val_x)
                    #val_loss = loss(val_output, val_y)
                    val_accuracy = torch.mean((torch.argmax(val_output, dim=1) == val_y).float())
                    try:
                        validation_accuracy_per_batch = torch.cat((validation_accuracy_per_batch ,val_accuracy))
                    except:
                        validation_accuracy_per_batch = val_accuracy
            #print('Validation Accuracy: {}'.format(validation_accuracy_per_batch.mean()))
            cnn_3_dict['val_acc_curve'].append(validation_accuracy_per_batch.mean().item())
            model_cnn_3.train()



# test the model
correct = 0
total = 0
for x, y in test_generator:
    output = model_cnn_3(x)
    _, predicted = torch.max(output, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print('Test Accuracy: {} %'.format(100 * correct / total))
cnn_3_dict['test_acc'] = 100 * correct / total
# save the model as a pty file
torch.save(model_cnn_3.state_dict(), 'q3_models/model_cnn_3.pty')
print("model saved as 'q3_models/model_cnn_3.pty'")
# get the parameters 784x32 layer as numpy array
weights_first_layer = model_cnn_3.Cnn1.weight.data.numpy()
cnn_3_dict['weights'] = weights_first_layer

# save cnn_3 as a pickle file
import pickle
with open('q3_models/cnn_3_dict.pkl', 'wb') as f:
    pickle.dump(cnn_3_dict, f)

### CNN 4


In [ ]:
%reset -f

# Load fashion MNIST dataset
import torchvision

from tqdm import tqdm

#training set
train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False, transform= torchvision.transforms.ToTensor())

#test set
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False, transform= torchvision.transforms.ToTensor())


import torch
# divide training data into training and validation sets of 0.8 and 0.2 respectively

train_data, val_data = torch.utils.data.random_split(train_data, [0.8, 0.2])

train_generator = torch.utils.data.DataLoader(train_data, batch_size=96, shuffle=True)
val_generator = torch.utils.data.DataLoader(val_data, batch_size=96, shuffle=False)
test_generator = torch.utils.data.DataLoader(test_data, batch_size=96, shuffle=False)





# example cnn_4 classifier
class cnn_4(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(cnn_4, self).__init__()
        self.input_size = input_size
        self.Conv1 = torch.nn.Conv2d(1 ,16 ,3, stride=1, padding=1)
        self.Conv2 = torch.nn.Conv2d(16, 8, 3,  stride=1, padding=1)
        self.Conv3 = torch.nn.Conv2d(8, 16, 5, stride=1, padding=1)
        self.MaxPool = torch.nn.MaxPool2d(2, stride=2)
        self.Conv4 = torch.nn.Conv2d(16, 16, 5,  stride=1, padding=1)
        self.prediction_layer = torch.nn.Linear(400, num_classes)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        hidden1 = self.Conv1(x)
        relu1 = self.relu(hidden1)
        hidden2 = self.Conv2(relu1)
        relu2 = self.relu(hidden2)
        hidden3 = self.Conv3(relu2)
        relu3 = self.relu(hidden3)
        pool1 = self.MaxPool(relu3)
        hidden4 = self.Conv4(pool1)
        pool2 = self.MaxPool(hidden4)
        flattened = pool2.view(96, -1)
        output = self.prediction_layer(flattened)
        return output

# initialize your model
model_cnn_4 = cnn_4(784 ,10)
if torch.cuda.is_available():
    model_cnn_4 = model_cnn_4.cuda()


# create loss: use cross entropy loss
loss = torch.nn.CrossEntropyLoss()
# create optimizer
optimizer = torch.optim.Adam(model_cnn_4.parameters(), lr=0.001)
# transfer your model to train mode
model_cnn_4.train()


cnn_4_dict = {"name":"cnn_4", "loss_curve": [],  "train_acc_curve": [], "val_acc_curve": [], "test_acc": 0, 'weights': []}



# train the model and save the training loss and validation loss for every 10 batches using model eval mode.s
for epoch in tqdm(range(15)):
    for batch, (x, y) in enumerate(train_generator):
        # forward pass
        output = model_cnn_4(x)
        # compute loss
        loss_val = loss(output, y)
        # zero gradients
        optimizer.zero_grad()
        # backward pass
        loss_val.backward()
        # optimize
        optimizer.step()
        # print loss
        if batch % 10 == 0:
            model_cnn_4.eval()
            #print('Epoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, loss_val.item()))
            cnn_4_dict['loss_curve'].append(loss_val.item())
            training_accuracy = torch.mean((torch.argmax(output, dim=1) == y).float())
            cnn_4_dict['train_acc_curve'].append(training_accuracy.item())

            # validation loss   
            validation_accuracy_per_batch = torch.tensor([])
            with torch.no_grad():
                for val_x, val_y in val_generator:
                    val_output = model_cnn_4(val_x)
                    #val_loss = loss(val_output, val_y)
                    val_accuracy = torch.mean((torch.argmax(val_output, dim=1) == val_y).float())
                    try:
                        validation_accuracy_per_batch = torch.cat((validation_accuracy_per_batch ,val_accuracy))
                    except:
                        validation_accuracy_per_batch = val_accuracy
            #print('Validation Accuracy: {}'.format(validation_accuracy_per_batch.mean()))
            cnn_4_dict['val_acc_curve'].append(validation_accuracy_per_batch.mean().item())
            model_cnn_4.train()



# test the model
correct = 0
total = 0
for x, y in test_generator:
    output = model_cnn_4(x)
    _, predicted = torch.max(output, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print('Test Accuracy: {} %'.format(100 * correct / total))
cnn_4_dict['test_acc'] = 100 * correct / total
# save the model as a pty file
torch.save(model_cnn_4.state_dict(), 'q3_models/model_cnn_4.pty')
print("model saved as 'q3_models/model_cnn_4.pty'")
# get the parameters 784x32 layer as numpy array
weights_first_layer = model_cnn_4.Cnn1.weight.data.numpy()
cnn_4_dict['weights'] = weights_first_layer

# save cnn_4 as a pickle file
import pickle
with open('q3_models/cnn_4_dict.pkl', 'wb') as f:
    pickle.dump(cnn_4_dict, f)

### CNN 5


In [ ]:
%reset -f

# Load fashion MNIST dataset
import torchvision

from tqdm import tqdm

#training set
train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False, transform= torchvision.transforms.ToTensor())

#test set
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False, transform= torchvision.transforms.ToTensor())


import torch
# divide training data into training and validation sets of 0.8 and 0.2 respectively

train_data, val_data = torch.utils.data.random_split(train_data, [0.8, 0.2])

train_generator = torch.utils.data.DataLoader(train_data, batch_size=96, shuffle=True)
val_generator = torch.utils.data.DataLoader(val_data, batch_size=96, shuffle=False)
test_generator = torch.utils.data.DataLoader(test_data, batch_size=96, shuffle=False)





# example cnn_5 classifier
class cnn_5(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(cnn_5, self).__init__()
        self.input_size = input_size
        self.Conv1 = torch.nn.Conv2d(1 ,8 ,3, stride=1, padding=1)
        self.Conv2 = torch.nn.Conv2d(8, 16, 3,  stride=1, padding=1)
        self.Conv3 = torch.nn.Conv2d(16, 8, 3, stride=1, padding=1)
        self.Conv4 = torch.nn.Conv2d(8, 16, 3,  stride=1, padding=1)
        self.MaxPool = torch.nn.MaxPool2d(2, stride=2)
        self.Conv5 = torch.nn.Conv2d(16, 16, 3,  stride=1, padding=1)
        self.Conv6 = torch.nn.Conv2d(16, 8, 3,  stride=1, padding=1)
        self.prediction_layer = torch.nn.Linear(392, num_classes)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        hidden1 = self.Conv1(x)
        relu1 = self.relu(hidden1)
        hidden2 = self.Conv2(relu1)
        relu2 = self.relu(hidden2)
        hidden3 = self.Conv3(relu2)
        relu3 = self.relu(hidden3)
        hidden4 = self.Conv4(relu3)
        relu4 = self.relu(hidden4)
        pool1 = self.MaxPool(relu4)
        hidden5 = self.Conv5(pool1)
        relu5 = self.relu(hidden5)
        hidden6 = self.Conv6(relu5)
        relu6 = self.relu(hidden6)
        pool2 = self.MaxPool(relu6)
        flattened = pool2.view(96, -1)
        output = self.prediction_layer(flattened)
        return output

# initialize your model
model_cnn_5 = cnn_5(784 ,10)
if torch.cuda.is_available():
    model_cnn_5 = model_cnn_5.cuda()


# create loss: use cross entropy loss
loss = torch.nn.CrossEntropyLoss()
# create optimizer
optimizer = torch.optim.Adam(model_cnn_5.parameters(), lr=0.001)
# transfer your model to train mode
model_cnn_5.train()


cnn_5_dict = {"name":"cnn_5", "loss_curve": [],  "train_acc_curve": [], "val_acc_curve": [], "test_acc": 0, 'weights': []}



# train the model and save the training loss and validation loss for every 10 batches using model eval mode.s
for epoch in tqdm(range(15)):
    for batch, (x, y) in enumerate(train_generator):
        # forward pass
        output = model_cnn_5(x)
        # compute loss
        loss_val = loss(output, y)
        # zero gradients
        optimizer.zero_grad()
        # backward pass
        loss_val.backward()
        # optimize
        optimizer.step()
        # print loss
        if batch % 10 == 0:
            model_cnn_5.eval()
            #print('Epoch: {}, Batch: {}, Loss: {}'.format(epoch, batch, loss_val.item()))
            cnn_5_dict['loss_curve'].append(loss_val.item())
            training_accuracy = torch.mean((torch.argmax(output, dim=1) == y).float())
            cnn_5_dict['train_acc_curve'].append(training_accuracy.item())

            # validation loss   
            validation_accuracy_per_batch = torch.tensor([])
            with torch.no_grad():
                for val_x, val_y in val_generator:
                    val_output = model_cnn_5(val_x)
                    #val_loss = loss(val_output, val_y)
                    val_accuracy = torch.mean((torch.argmax(val_output, dim=1) == val_y).float())
                    try:
                        validation_accuracy_per_batch = torch.cat((validation_accuracy_per_batch ,val_accuracy))
                    except:
                        validation_accuracy_per_batch = val_accuracy
            #print('Validation Accuracy: {}'.format(validation_accuracy_per_batch.mean()))
            cnn_5_dict['val_acc_curve'].append(validation_accuracy_per_batch.mean().item())
            model_cnn_5.train()



# test the model
correct = 0
total = 0
for x, y in test_generator:
    output = model_cnn_5(x)
    _, predicted = torch.max(output, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print('Test Accuracy: {} %'.format(100 * correct / total))
cnn_5_dict['test_acc'] = 100 * correct / total
# save the model as a pty file
torch.save(model_cnn_5.state_dict(), 'q3_models/model_cnn_5.pty')
print("model saved as 'q3_models/model_cnn_5.pty'")
# get the parameters 784x32 layer as numpy array
weights_first_layer = model_cnn_5.Cnn1.weight.data.numpy()
cnn_5_dict['weights'] = weights_first_layer

# save cnn_5 as a pickle file
import pickle
with open('q3_models/cnn_5_dict.pkl', 'wb') as f:
    pickle.dump(cnn_5_dict, f)

## Experimenting Activation Functions

## Experimenting Learning Rate